In [ ]:
import boto3
import random

s3client = boto3.client('s3')

In [ ]:
# Bucket names are globally unique. Here we'll add a random element to the name to increase the odds of
# avoiding a name collision
bucket_name = 'ds-aussie-' + str(random.randint(1, 9999999))
print bucket_name

In [ ]:
response = s3client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={
         'LocationConstraint':'ap-southeast-2'
    }
)

print response

In [ ]:
# Upload content
s3 = boto3.resource('s3')
response = s3.meta.client.upload_file('./aws-art.png', bucket_name, 'aws-art.png')
print response

In [ ]:
# Make the object public readable
object_acl = s3.ObjectAcl(bucket_name,'aws-art.png')
response = object_acl.put(
    ACL='public-read'
)

print response

In [ ]:
# Create the origin access identity we'll use for accessing the s3 bucket via cf
cf_client = boto3.client('cloudfront')
origin_access_id = cf_client.create_cloud_front_origin_access_identity(
    CloudFrontOriginAccessIdentityConfig={
        'CallerReference': bucket_name,
        'Comment': 'Identity for bucket access via CF'
    }
)

print origin_access_id

extracted_origin_access_id = origin_access_id['CloudFrontOriginAccessIdentity']['Id']

print extracted_origin_access_id


In [ ]:
# Create a cloud front web distribution 
response = cf_client.create_distribution(
     DistributionConfig={
        'Comment':'For fun and profit',
        'Enabled':True,
        'CallerReference':bucket_name,
        'Origins': {
            'Quantity':1,
            'Items': [
                {
                    'Id':'S3-' + bucket_name,
                    'DomainName': bucket_name + '.s3.amazonaws.com',
                    'S3OriginConfig': {
                        'OriginAccessIdentity': 'origin-access-identity/cloudfront/' + extracted_origin_access_id
                    }
                }
            ]
        },
        'DefaultCacheBehavior': {
            'TargetOriginId': 'S3-' + bucket_name,
            'ForwardedValues': {
                'QueryString': True,
                'Cookies': {
                    'Forward': 'all'
                }
            },
            'TrustedSigners': {
                'Enabled': False,
                'Quantity': 0
            },
            'ViewerProtocolPolicy': 'allow-all',
            'MinTTL':86400
        }
    }
)

print response

In [ ]:
distribution_id = response['Distribution']['Id']
print distribution_id

response = cf_client.update_distribution(
     DistributionConfig={
        'Comment':'disable prior to delete',
        'DefaultRootObject':'',
        'WebACLId':'',
        'Aliases': {
            'Quantity': 0
        },
        'PriceClass':'PriceClass_All',
        'Enabled':False,
        'CallerReference':bucket_name,
        'Origins': {
            'Quantity':0
        },
        'DefaultCacheBehavior': {
            'TargetOriginId': 'S3-' + bucket_name,
            'ForwardedValues': {
                'QueryString': True,
                'Cookies': {
                    'Forward': 'all'
                }
            },
            'TrustedSigners': {
                'Enabled': False,
                'Quantity': 0
            },
            'ViewerProtocolPolicy': 'allow-all',
            'MinTTL':86400
        },
        'Logging': {
            'Enabled': False,
            'IncludeCookies':False,
            'Bucket': '',
            'Prefix': ''
        }
    },
    
    Id=distribution_id,
    IfMatch=distribution_id

)

print response

In [ ]:
distribution_id = response['Distribution']['Id']
print distribution_id

create_etag = response['ETag']

config_response = cf_client.get_distribution_config(
    Id=distribution_id
)

config = config_response['DistributionConfig']
print config

In [ ]:
config['Enabled'] = False
disable_response = cf_client.update_distribution(
    Id=distribution_id,
    IfMatch=create_etag,
    DistributionConfig=config
)

print disable_response

In [ ]:
disable_etag = disable_response['ETag']
print disable_etag

delete_response = cf_client.delete_distribution(
    Id=distribution_id,
    IfMatch=disable_etag
)

print delete_response

In [ ]:
# Delete bucket content
response = s3client.delete_object(
        Bucket=bucket_name,
        Key='aws-art.png'
)

print response

In [ ]:
response = s3client.delete_bucket(
    Bucket=bucket_name
)

print response